In [55]:
import os
import pandas as pd
import numpy as np
import re
import time
from datetime import datetime
from datetime import timedelta

# Prepare Olist

In [52]:
%%time
PATH_OLIST='../../data/olist/'
df_customers=pd.read_parquet(f'{PATH_OLIST}olist_customers_dataset.parquet')
df_order_items=pd.read_parquet(f'{PATH_OLIST}olist_order_items_dataset.parquet')
df_orders=pd.read_parquet(f'{PATH_OLIST}olist_orders_dataset.parquet')
df_products=pd.read_parquet(f'{PATH_OLIST}olist_products_dataset.parquet')
df_sellers=pd.read_parquet(f'{PATH_OLIST}olist_sellers_dataset.parquet')

CPU times: total: 453 ms
Wall time: 300 ms


In [53]:
df_olist=df_order_items.merge(
    df_orders.merge(df_customers,
                    how='left',
                    left_on='customer_id',
                    right_on='customer_id',
    ),
    how='left',
    left_on='order_id',
    right_on='order_id',
).merge(df_products,
        how='left',
        left_on='product_id',
        right_on='product_id',        
).merge(df_sellers,
        how='left',
        left_on='seller_id',
        right_on='seller_id',  
       )
print(f'Error - new duplicate rows: {len(df_order_items)-len(df_olist)}')

df_olist['date_sale']= pd.to_datetime(df_olist['order_purchase_timestamp'])
df_olist.sort_values(by=['date_sale'], inplace=True)
df_olist['sales']=df_olist['price']*df_olist['order_item_id']
df_olist['sales_full']=df_olist['freight_value']*df_olist['order_item_id']+df_olist['sales']
df_olist.to_parquet(f'{PATH_OLIST}olist.parquet')
df_olist.head(5)

Error - new duplicate rows: 0


,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,customer_id,order_status,order_purchase_timestamp,...,product_weight_g,product_length_cm,product_height_cm,product_width_cm,seller_zip_code_prefix,seller_city,seller_state,date_sale,sales,sales_full
20265,2e7a8482f6fb09756ca50c10d7bfc047,1,c1488892604e4ba5cff5b4eb4d595400,1554a68530182680ad5c8b042c3ab563,2016-10-26 18:25:19,39.99,31.67,08c5351a6aca1c1589a38f244edeee9d,shipped,2016-09-04 21:15:19,...,1400.0,32.0,6.0,28.0,37580,monte siao,MG,2016-09-04 21:15:19,39.99,71.66
20266,2e7a8482f6fb09756ca50c10d7bfc047,2,f293394c72c9b5fafd7023301fc21fc2,1554a68530182680ad5c8b042c3ab563,2016-10-26 18:25:19,32.90,31.67,08c5351a6aca1c1589a38f244edeee9d,shipped,2016-09-04 21:15:19,...,1800.0,32.0,6.0,28.0,37580,monte siao,MG,2016-09-04 21:15:19,65.80,129.14
101394,e5fa5a7210941f7d56d0208e4e071d35,1,f3c2d01a84c947b078e32bbef0718962,a425f92c199eb576938df686728acd20,2016-09-19 00:15:34,59.50,15.56,683c54fc24d40ee9f8a6fc179fd9856c,canceled,2016-09-05 00:15:34,...,700.0,25.0,2.0,25.0,81050,curitiba,PR,2016-09-05 00:15:34,59.50,75.06
84389,bfbd0f9bdef84302105ad712db648a6c,1,5a6b04657a4c5ee34285d1e4619a96b4,ecccfa2bb93b34a3bf033cc5d1dcdc69,2016-09-19 23:11:33,44.99,2.83,86dc2ffce2dfff336de2f386a786e574,delivered,2016-09-15 12:16:38,...,1000.0,16.0,16.0,16.0,81810,curitiba,PR,2016-09-15 12:16:38,44.99,47.82
84391,bfbd0f9bdef84302105ad712db648a6c,3,5a6b04657a4c5ee34285d1e4619a96b4,ecccfa2bb93b34a3bf033cc5d1dcdc69,2016-09-19 23:11:33,44.99,2.83,86dc2ffce2dfff336de2f386a786e574,delivered,2016-09-15 12:16:38,...,1000.0,16.0,16.0,16.0,81810,curitiba,PR,2016-09-15 12:16:38,134.97,143.46


# Prepare TGE

In [56]:
%%time
PATH_TGE='../../data/tge/'
file_name='tge'
df=pd.read_parquet(f'{PATH_TGE}{file_name}.parquet')
df.sort_values(by=['date','czas'], inplace=True)
def create_datetime(d,h):
    return datetime.strptime(d, "%Y-%m-%d") + timedelta(hours=(h-1))
df['date_hour']=df.apply(lambda x:create_datetime(x.date,x.czas), axis=1)
df.index=df['date_hour']
df.to_parquet(f'{PATH_TGE}tge_full.parquet')
df.head()

CPU times: total: 703 ms
Wall time: 703 ms


,date,czas,fixing_1_kurs,fixing_1_wolumen,fixing_2_kurs,fixing_2_wolumen,notowania_kurs,notowania_wolumen,date_hour
date_hour,,,,,,,,,
2019-06-28 00:00:00,2019-06-28,1,220.54,1754.3,226.53,1245.7,0.0,5.0,2019-06-28 00:00:00
2019-06-28 01:00:00,2019-06-28,2,203.85,1674.0,191.84,1277.3,0.0,0.0,2019-06-28 01:00:00
2019-06-28 02:00:00,2019-06-28,3,203.85,1786.2,201.12,1079.6,0.0,0.0,2019-06-28 02:00:00
2019-06-28 03:00:00,2019-06-28,4,203.85,1689.4,211.05,844.5,0.0,0.0,2019-06-28 03:00:00
2019-06-28 04:00:00,2019-06-28,5,203.85,1793.9,222.50,602.3,0.0,0.0,2019-06-28 04:00:00


# Growth and Increase

In [39]:
df['increase']=df['fixing_1_kurs'].sub(df['fixing_1_kurs'].shift(1))
df['growth_rate_prc']=df['fixing_1_kurs'].div(df['fixing_1_kurs'].shift(1)).sub(1).mul(100)
df['increase_accumulated']=df['fixing_1_kurs'].sub(df['fixing_1_kurs'].iloc[0])
df['growth_rate_accumulated_prc']=df['fixing_1_kurs'].div(df['fixing_1_kurs'].iloc[0]).sub(1).mul(100) 
df['gra_ma_month_prc']=df['fixing_1_kurs'].div(df['fixing_1_kurs'].iloc[0]).sub(1).mul(100).rolling(30).mean() # moving_average

In [40]:
df

,date,czas,fixing_1_kurs,fixing_1_wolumen,fixing_2_kurs,fixing_2_wolumen,notowania_kurs,notowania_wolumen,increase,growth_rate_prc,increase_accumulated,growth_rate_accumulated_prc,gra_ma_month_prc
29312,2019-06-28,1,220.54,1754.3,226.53,1245.7,0.0,5.0,NaN,NaN,0.00,0.000000,NaN
29313,2019-06-28,2,203.85,1674.0,191.84,1277.3,0.0,0.0,-16.69,-7.567788,-16.69,-7.567788,NaN
29314,2019-06-28,3,203.85,1786.2,201.12,1079.6,0.0,0.0,0.00,0.000000,-16.69,-7.567788,NaN
29315,2019-06-28,4,203.85,1689.4,211.05,844.5,0.0,0.0,0.00,0.000000,-16.69,-7.567788,NaN
29316,2019-06-28,5,203.85,1793.9,222.50,602.3,0.0,0.0,0.00,0.000000,-16.69,-7.567788,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19,2022-10-31,20,585.01,1485.8,598.39,333.3,0.0,0.0,-27.50,-4.489723,364.47,165.262537,159.203017
20,2022-10-31,21,550.96,1328.6,565.81,301.3,550.0,23.0,-34.05,-5.820413,330.42,149.823161,157.856171
21,2022-10-31,22,480.96,1339.9,499.80,311.3,0.0,0.0,-70.00,-12.705097,260.42,118.082887,153.562921
22,2022-10-31,23,430.00,1676.9,465.88,291.8,0.0,0.0,-50.96,-10.595476,209.46,94.975968,147.970587


# Сolumn <=> multiple columns

In [ ]:
# from columns to 'each cell contains a list'
list_of_columns=['col1','col2','col3']
df['col_list'] = df_pay[list_of_columns].values.tolist()

In [ ]:
# from 'each cell contains a list' to columns
df=pd.concat([df, pd.DataFrame(df['col_list'].tolist(), columns=['col1','col2','col3'])], axis=1)

In [ ]:
# from 'each cell contains a list' to string
df['col_str'] = df['col_list'].map(lambda x: ', '.join(x))

In [ ]:
# from string to columns
df=pd.concat([df,df['col_str'].str.split(', ', expand=True).rename(columns={0: 'col1', 1: 'col2'})],axis=1)

In [ ]:
# select one value from list in cells
df['col_uniq']=df['col_list'].map(lambda x: x[0] if len(x)>0 else None)

In [42]:
# from json in cell to columns
# sample DataFrame
df = pd.DataFrame({
    "data": [
        '{"key1": "value1", "key2": "value2"}',
        '{"key1": "value3", "key2": "value4"}'
    ]
})

# from JSON into multiple columns
df = df.join(df.pop("data").apply(pd.io.json.loads).apply(pd.Series))

print(df)

     key1    key2
0  value1  value2
1  value3  value4


In [ ]:
# from columns to dict (index: col_str)
dict_col=df['col_str'].to_dict()
df['col_dict']=df['col_str'].to_dict()

In [ ]:
# from dict to columns
df["col_str_from_dict"] = df["index"].map(dict_col)